<a href="https://colab.research.google.com/github/NID123-CH/DSR-40/blob/main/DSR_40__feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import matplotlib.pyplot as plt
from scipy.stats import skewnorm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import spacy
from sklearn.preprocessing import LabelEncoder

from sklearn.cluster import KMeans


# Feature engineering

Adding columns or transform existing columns
- this includes the target (target transformation)

## How can I generate features?

Domain knowledge
- use business insight -> features

Exploratory data analysis

Build features from features
- ratios
- polynomials
- log / exponentials / sigmoids
- Fourier / Laplace transforms

## Data leakage

Feature available during training that isn't available during testing / production time

Duplicates in the train & test set

Training / learning statistics from the test set
- common mistake to refit normalization / standardization

Feature engineering can often cause data leakage
- mean target encoding on the entire dataset, before splitting test & train

## Nominal vs ordinal

Nominal (no order) versus ordinal (order)

Some methods place order onto a nominal series during the encoding

## Transform from continuous to categorical

On the target
- moving from a regression to classification problem
- key thing to ask - how do we use our prediction in our business problem

On features
- removes noise (& maybe signal)

In [10]:
from google.colab import files
files.upload()

Saving Sample.csv to Sample (2).csv


{'Sample (2).csv': b'customers-category,contract-length,location\r\nA,10,us\r\nA,9,us\r\nA,9,us\r\nB,20,nz\r\nB,30,NaN\r\nB,10,nz\r\nC,NaN,nz\r\nC,5,nz\r\nC,NaN,us\r\nNaN,1,nz\r\nC,100,nz\r\n'}

In [11]:
ds = pd.read_csv('Sample.csv')
print(ds.columns)

Index(['id', 'customers-category', 'contract-length', 'location'], dtype='object')


In [12]:
ds

,id,customers-category,contract-length,location
0,0,A,10.0,us
1,1,A,9.0,us
2,2,A,9.0,us
3,3,B,20.0,nz
4,4,B,30.0,NaN
5,5,B,10.0,nz
6,6,C,NaN,nz
7,7,C,5.0,nz
8,8,C,NaN,us
9,9,NaN,1.0,nz


In [ ]:
#Pandas cut() function is used to separate the array elements into different bins . The cut function is mainly used to perform statistical analysis on scalar data.
pd.cut(ds.loc[:, 'contract-length'].fillna(0), bins=4)

0      (-0.1, 25.0]
1      (-0.1, 25.0]
2      (-0.1, 25.0]
3      (-0.1, 25.0]
4      (25.0, 50.0]
5      (-0.1, 25.0]
6      (-0.1, 25.0]
7      (-0.1, 25.0]
8      (-0.1, 25.0]
9      (-0.1, 25.0]
10    (75.0, 100.0]
Name: contract-length, dtype: category
Categories (4, interval[float64, right]): [(-0.1, 25.0] < (25.0, 50.0] < (50.0, 75.0] < (75.0, 100.0]]

## Dealing with categorical variables

Machine always think in numbers
- all categorical features need to be transformed into numbers at somepoint
- most ML algorithms get upset if you don't do it - some will do it on the fly

## One hot encoding

- curse of dimensionality makes dimensionality increase exponential
- lose the explicit relationship of the feature (model now just sees a lot of columns, and has to learn their relationship)

There are a few ways to do one-hot encoding in the Python
- `sklearn.preprocessing.OneHotEncoder()`
- `pd.get_dummies()`

Recommend `sklearn.preprocessing.OneHotEncoder()`
- creates a stateful transformer that can be reused (i.e. on the test data)
- using `pd.get_dummies`, you are hoping that the columns will be encoded in the same way at test time

In [15]:
import pandas as pd

con = pd.Series(list('abcba'))
print(pd.get_dummies(con))

       a      b      c
0   True  False  False
1  False   True  False
2  False  False   True
3  False   True  False
4   True  False  False


In [16]:
# One-Hot Encoding a Single DataFrame Series
import pandas as pd

df = pd.DataFrame({
    'Name': ['Joan', 'Matt', 'Jeff', 'Melissa', 'Devi'],
    'Gender': ['Female', 'Male', 'Male', 'Female', 'Female'],
    'House Type': ['Apartment', 'Detached', 'Apartment', None, 'Semi-Detached']
    })

print(pd.get_dummies(df['Gender']))

   Female   Male
0    True  False
1   False   True
2   False   True
3    True  False
4    True  False


In [17]:
from sklearn.preprocessing import OneHotEncoder

#  note that I use a missing value token here
cat = ds.loc[:, 'customers-category'].to_frame().fillna('missing')

#  will by default return a sparse matrix - you can turn this off
enc = OneHotEncoder(sparse=False)
enc.fit_transform(cat)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

## Label encoding

- `0, 1, 2, 3`
- is an ordinal encoding - even if feature is not ordinal, you are imposing this structure on the data

Decision trees sort the data based on the feature being split on
- this makes the decision boundary meaningful
- inherently ordinal

In [18]:
enc = LabelEncoder()
enc.fit_transform(cat)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 2])

## Mean encode

- put the training data average for the target for that class
- could also use other statistics like median, quantiles or variance

The code below will mean encode a categorical feature using the average of another column (often the target):

In [19]:
store1 = pd.DataFrame(
        {'store': ['A'] * 3,
         'Sales': [100, 200, 300],
         'noise': [0, 0, 0]}
    )

store2 = pd.DataFrame(
        {'store': ['B'] * 3,
         'Sales': [10, 20, 30],
         'noise': [0, 0, 0]}
    )

data = pd.concat([store1, store2], axis=0)
data

,store,Sales,noise
0,A,100,0
1,A,200,0
2,A,300,0
0,B,10,0
1,B,20,0
2,B,30,0


In [21]:
def mean_encode(data, col, on):
    # Convert the column to numeric if it's of object type
    if data[col].dtype == 'object':
        data[col] = pd.to_numeric(data[col], errors='coerce')
    group = data.groupby(col).mean()
    data.loc[:, col+'-original'] = data.loc[:, col]
    mapper = {k: v for k, v in zip(group.index, group.loc[:, on].values)}

    data.loc[:, col] = data.loc[:, col].replace(mapper)
    data.loc[:, col].fillna(value=np.mean(data.loc[:, col]), inplace=True)
    return data



def test_mean_encoding():
    store1 = pd.DataFrame(
        {'store': ['A'] * 3,
         'Sales': [100, 200, 300],
         'noise': [0, 0, 0]}
    )

    store2 = pd.DataFrame(
        {'store': ['B'] * 3,
         'Sales': [10, 20, 30],
         'noise': [0, 0, 0]}
    )

    data = pd.concat([store1, store2], axis=0)
    data = mean_encode(data, col='store', on='Sales')
    np.testing.assert_array_equal(
        data.loc[:, 'store'], np.array([200, 200, 200, 20, 20, 20])
    )

test_mean_encoding()
data = mean_encode(ds.copy(), 'customers-category', 'contract-length')
data.drop(['contract-length', 'location'], axis=1)

AssertionError: 
Arrays are not equal

x and y nan location mismatch:
 x: array([nan, nan, nan, nan, nan, nan])
 y: array([200, 200, 200,  20,  20,  20])

## Binary encoding

Encoding the string using it's binary (0110100 etc) representation

In Python we can do this using the `bin()` builtin:

In [30]:
bin(4)

'0b100'

In [31]:
bin(5)

'0b101'

The `category_encoders` library can do this for us:

In [33]:
!pip install category_encoders
import category_encoders as ce # import the category_encoders library and alias it as 'ce'
import pandas as pd

data = ['A', 'BB', 'B', 'A', 'C', 'D', 'A', 'E']

enc = ce.BinaryEncoder()  #similar to onehot, but stores categories as binary bitstrings.

print('{} classes'.format(len(set(data))))

pd.concat([enc.fit_transform(data), pd.DataFrame(data, columns=['data'])], axis=1)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.7 MB/s eta 0:00:00
6 classes


,0_0,0_1,0_2,data
0,0,0,1,A
1,0,1,0,BB
2,0,1,1,B
3,0,0,1,A
4,1,0,0,C
5,1,0,1,D
6,0,0,1,A
7,1,1,0,E


## Frequency encoding

Encode the categorical feature based on their relative frequency
- probability of seeing this category
- tell the model about rare categories
- but you can't distinguish categories with the same frequency (unlikely in very large datasets)

In [34]:
data = ['A', 'B', 'A', 'C', 'D', 'A', 'E']
counter = Counter(data)
freq_enc = [counter[x] / len(data) for x in data]

pd.DataFrame(
    {'data': data,
     'freq_enc': freq_enc}
)

,data,freq_enc
0,A,0.428571
1,B,0.142857
2,A,0.428571
3,C,0.142857
4,D,0.142857
5,A,0.428571
6,E,0.142857


### Others

- hint your model as much as possible.

In [35]:
data = pd.DataFrame(
    {'sales': [10, 50, 100], 'year': [2012, 2013, 2014]}
)

data

,sales,year
0,10,2012
1,50,2013
2,100,2014


In [36]:
# New Feature Generation #
data.loc[:, 'last-year-sales-feature'] = data.loc[:, 'sales'].shift()
data.loc[:, 'model-target'] = data.loc[:, 'sales'].diff()

data

,sales,year,last-year-sales-feature,model-target
0,10,2012,NaN,NaN
1,50,2013,10.0,40.0
2,100,2014,50.0,50.0


## Dimensionality reduction

### PCA

Both forms of dimensionality reduction, that can create new columns
- these can sit alongside or replace the high dimensional data

### Clustering

Can be used to create a new feature:

In [27]:
data = pd.DataFrame([np.random.uniform(0, x, 1000) for x in range(1, 10)]).T

mdl = KMeans(3)
mdl.fit(data)
data.loc[:, 'cluster'] = mdl.labels_

data.head()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,0,1,2,3,4,5,6,7,8,cluster
0,0.086822,1.135188,0.752935,2.281399,4.800599,2.229931,1.970671,1.036777,2.128022,1
1,0.197899,0.763622,2.453104,2.199509,4.131537,5.877712,6.394513,3.750568,5.875380,0
2,0.685786,1.072281,1.298486,1.431405,1.908599,3.349460,2.755600,7.099060,6.852797,2
3,0.759410,0.702483,0.627350,1.288103,3.951796,0.483573,0.683995,1.550884,6.760608,0
4,0.826652,0.196725,2.162732,0.076861,1.818472,3.621377,1.459697,7.586151,5.936767,2


## Datetime

### Month, year, day columns

These can then be either be label / one-hot encoded

In [37]:
data = pd.DataFrame(
    {'date': pd.date_range('01-01-2018', '02-01-2018', freq='1h')}
)

data.loc[:, 'month'] = data.loc[:, 'date'].dt.month
data.loc[:, 'minute'] = data.loc[:, 'date'].dt.minute

data.head()

,date,month,minute
0,2018-01-01 00:00:00,1,0
1,2018-01-01 01:00:00,1,0
2,2018-01-01 02:00:00,1,0
3,2018-01-01 03:00:00,1,0
4,2018-01-01 04:00:00,1,0


## Time series decomposition

Seasonality, trend can become features

More important than other areas to ask the question
- what will I have available at test time?
- example of weather versus weather forecasts

## References

Gabby Shklovsky - Random Forests Best Practices for the Business World - PyData NYC 2017 [youtube](https://www.youtube.com/watch?v=E7VLE-U07x0) - [slides](https://www.youtube.com/redirect?q=https%3A%2F%2Fwww.slideshare.net%2FPyData%2Frandom-forests-best-practices-for-the-business-world&redir_token=HgV_RBYb_uD_jYV6nYygn8RpyKR8MTU2OTkwODE2N0AxNTY5ODIxNzY3&v=E7VLE-U07x0&event=video_description)

Art of Feature Engineering for Data Science - Nabeel Sarwar - [youtube](https://youtu.be/leTyvBPhYzw)

Feature Engineering with H2O - Dmitry Larko, Senior Data Scientist, H2O.ai - [youtube](https://youtu.be/irkV4sYExX4)

[Building Machine Learning Powered Applications: Going from Idea to Product](https://www.amazon.com/Building-Machine-Learning-Powered-Applications/dp/149204511X)

[Datacamp categorical data tutorial](https://www.datacamp.com/community/tutorials/categorical-data)

[Smarter Ways to Encode Categorical Data for Machine Learning](https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159)

[Vincent Warmerdam: Winning with Simple, even Linear, Models | PyData London 2018](https://www.youtube.com/watch?v=68ABAU_V8qI)

[Why giving your algorithm ALL THE FEATURES does not always work - Thomas Huijskens](https://www.youtube.com/watch?v=JsArBz46_3s)

[Feature Engineering - Elite Data Science](https://elitedatascience.com/feature-engineering)

[Feature Engineering for ML](https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114)

See *Feature Engineering with H2O - Dmitry Larko, Senior Data Scientist, H2O.ai - [youtube](https://youtu.be/irkV4sYExX4)*


*Gabby Shklovsky - Random Forests Best Practices for the Business World - PyData NYC 2017 [youtube](https://www.youtube.com/watch?v=E7VLE-U07x0) - [slides](https://www.youtube.com/redirect?q=https%3A%2F%2Fwww.slideshare.net%2FPyData%2Frandom-forests-best-practices-for-the-business-world&redir_token=HgV_RBYb_uD_jYV6nYygn8RpyKR8MTU2OTkwODE2N0AxNTY5ODIxNzY3&v=E7VLE-U07x0&event=video_description)*

*Gabby Shklovsky - Random Forests Best Practices for the Business World - PyData NYC 2017 [youtube](https://www.youtube.com/watch?v=E7VLE-U07x0) - [slides](https://www.youtube.com/redirect?q=https%3A%2F%2Fwww.slideshare.net%2FPyData%2Frandom-forests-best-practices-for-the-business-world&redir_token=HgV_RBYb_uD_jYV6nYygn8RpyKR8MTU2OTkwODE2N0AxNTY5ODIxNzY3&v=E7VLE-U07x0&event=video_description)*